In [1]:
import sys
import time
import gzip
from concurrent import futures
from functools import partial
import pandas as pd
from CHARMio import parse_pairs, parse_gtf, write_pairs

In [2]:
def cli(args)->int:
    filename, gtf_file, out_name, thread = \
        args.filename[0], args.gtf_filename, args.out_name, args.num_thread
    # parsing ref gtf and pairs file
    pairs = parse_pairs(filename)
    # build in-mem exon index
    gtf = parse_gtf(gtf_file)
    ref = build_in_memory_index(get_exon(gtf))
    # do search
    cleaned = clean_splicing(pairs, ref, thread)
    write_pairs(cleaned, out_name)
def get_exon(gtf:pd.DataFrame) -> pd.DataFrame:
    # extract exon-gene_name from gtf table
    relevant = gtf.query('feature == "exon" & source == "HAVANA"') #using HAVANA only
    gene_id = relevant["group"].str.extract('gene_id "(ENSG[0-9]{11}.[0-9])";') #extract gene name from group
    gene_id.columns = ["gene_id"] # extract returns dataframe rather than series
    # don't mind strand
    return pd.concat([relevant.drop(["group","feature","source","score","strand","frame"],axis=1),gene_id],axis=1)
 
def build_in_memory_index(exons:pd.DataFrame) -> dict:
    # split by chr and using IntervalIndex to enable searching
    ref_dict = {key : value for key, value in exons.groupby("seqname")}
    # build index by chromosome
    for chromosome in ref_dict:
        # using start, end attrs as index
        by_chr_table = ref_dict[chromosome]
        bed_tuple = by_chr_table.set_index(['start','end']).index 
        bed_interval_index = pd.IntervalIndex.from_tuples(bed_tuple)
        by_chr_table.index = bed_interval_index
        ref_dict[chromosome] = by_chr_table.drop(["start","end","seqname"],axis=1)
    sys.stderr.write("hires_utils::clean_splicing: index done.\n")
    return ref_dict
def legs_co_gene(contact:pd.Series, chromosome:str, ref_dict:dict)->bool:
    # whether two legs of contacts are in same gene's exon
    # must be intra contacts
    pos1, pos2 = contact["pos1"], contact["pos2"]
    result = ref_dict[chromosome][ref_dict[chromosome].index.contains(pos1)]
    if len(result[result.index.contains(pos2)]) > 0:
        return True
    else:
        return False
def search_chromosome(contacts_at_chromosome:tuple, ref:dict)->pd.DataFrame:
    # search whole chromosome using F::legs_co_gene
    # single chr searching for multi-process calling
    chromosome, contacts = contacts_at_chromosome[0], contacts_at_chromosome[1]
    hit_index = contacts.apply(legs_co_gene, chromosome=chromosome, ref_dict=ref, axis=1)
    return contacts[hit_index]

def clean_splicing(pairs:pd.DataFrame, ref:dict, thread:int)->pd.DataFrame:
    '''
    clean contacts from splicing
    '''
    #intra = pairs.query('chr1 == chr2') # only search for intra
    intra = pairs.loc[pairs['chr1'] == pairs['chr2']]
    working_func = partial(search_chromosome, ref=ref) # pool.map can't take multiple iterable as arguments
    input_data = [(chromosome, per_chr_contacts) for chromosome, per_chr_contacts in intra.groupby("chr1")] # pool.map can't take additional keyword argument
    sys.stderr.write("hires_utils::clean_splicing: input parsed, search in %d thread\n" % thread)
    # do multi-thread searching
    with futures.ProcessPoolExecutor(thread) as pool:
        res = pool.map(working_func, input_data)
    result = pd.concat(res, axis=0) # contained contacts
    cleaned = pairs.drop(result.index, axis=0) # clean contacts
    print("clean_splicing: %d contacts removed in %s\n" %(len(result), pairs.attrs["name"]) )
    return cleaned

In [4]:
gtf_file = "/share/Data/public/ref_genome/rabbit_ref/rawdata/genomic.gtf"
gtf_1 = "/share/Data/public/ref_genome/human/rawdata/genomic.gtf"
gtf = parse_gtf(gtf_file)

In [20]:
import re

In [22]:
re.search(r'rabbit', gtf_file).group(0)

'rabbit'

In [17]:
def get_exon_rab(gtf:pd.DataFrame) -> pd.DataFrame:
    # extract exon-gene_name from gtf table
    relevant = gtf.query('feature == "exon"') #using HAVANA only
    gene_id = relevant["group"].str.extract('gene_id "([A-Za-z0-9_-]+)";') #extract gene name from group
    gene_id.columns = ["gene_id"] # extract returns dataframe rather than series
    # don't mind strand
    return pd.concat([relevant.drop(["group","feature","source","score","strand","frame"],axis=1),gene_id],axis=1)

In [18]:
gtf

,seqname,source,feature,start,end,score,strand,frame,group
0,NC_067374.1,Gnomon,gene,4619,31619,.,+,.,"gene_id ""SH2D3C""; transcript_id """"; db_xref ""G..."
1,NC_067374.1,Gnomon,transcript,4619,31619,.,+,.,"gene_id ""SH2D3C""; transcript_id ""XM_051835448...."
2,NC_067374.1,Gnomon,exon,4619,4777,.,+,.,"gene_id ""SH2D3C""; transcript_id ""XM_051835448...."
3,NC_067374.1,Gnomon,exon,6191,6665,.,+,.,"gene_id ""SH2D3C""; transcript_id ""XM_051835448...."
4,NC_067374.1,Gnomon,exon,15918,15957,.,+,.,"gene_id ""SH2D3C""; transcript_id ""XM_051835448...."
...,...,...,...,...,...,...,...,...,...
2051336,NC_001913.1,RefSeq,transcript,15314,15379,.,+,.,"gene_id ""KEF51_t21""; transcript_id ""unassigned..."
2051337,NC_001913.1,RefSeq,exon,15314,15379,.,+,.,"gene_id ""KEF51_t21""; transcript_id ""unassigned..."
2051338,NC_001913.1,RefSeq,gene,15380,15445,.,-,.,"gene_id ""KEF51_t22""; transcript_id """"; db_xref..."
2051339,NC_001913.1,RefSeq,transcript,15380,15445,.,-,.,"gene_id ""KEF51_t22""; transcript_id ""unassigned..."


In [19]:
get_exon_rab(gtf)

,seqname,start,end,gene_id
2,NC_067374.1,4619,4777,SH2D3C
3,NC_067374.1,6191,6665,SH2D3C
4,NC_067374.1,15918,15957,SH2D3C
5,NC_067374.1,21189,21317,SH2D3C
6,NC_067374.1,22568,23019,SH2D3C
...,...,...,...,...
2051323,NC_001913.1,13579,14103,KEF51_p02
2051329,NC_001913.1,14104,14171,KEF51_t20
2051332,NC_001913.1,14175,15313,KEF51_p01
2051337,NC_001913.1,15314,15379,KEF51_t21
